In [1]:
#INSTALL LIBRARIES------------------------------------------
!pip install transformers scikit-learn datasets wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.1/302.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 1.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: py

In [23]:
#IMPORTS-----------------------------
from pprint import pprint
from datasets import load_dataset
from transformers import RobertaTokenizer, RobertaModel
from torch.utils.data import Dataset, DataLoader
import torch
import matplotlib.pyplot as plt
from sklearn.metrics import  confusion_matrix as compute_confusion_matrix, ConfusionMatrixDisplay
from torch.nn import Linear, ReLU
import pdb
import numpy as np, torch, random as rnd, torch.nn as nn, wandb as wndb
from torch.nn.utils.rnn import pad_sequence

In [3]:


adversarial = load_dataset("iperbole/adversarial_fever_nli")["test"]

ds = load_dataset("tommasobonomo/sem_augmented_fever_nli")

training_set = ds["train"]

validation_set = ds["validation"]

test_set = ds["test"]

pprint(adversarial[0])
pprint(training_set[0])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating test split:   0%|          | 0/337 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/51086 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2288 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2287 [00:00<?, ? examples/s]

{'cid': 58846,
 'hypothesis': 'The number of sitcoms from France in which Johnny Galecki has '
               'played a character is greater or equal to 2',
 'label': 'NEUTRAL',
 'part': 'manual_adversarial',
 'premise': 'Johnny Galecki . He is known for playing David Healy in the ABC '
            'sitcom Roseanne from 1992 -- 1997 and Dr. Leonard Hofstadter in '
            'the CBS sitcom The Big Bang Theory since 2007 .'}
{'hypothesis': 'Roman Atwood is a content creator.',
 'id': '150448',
 'label': 'ENTAILMENT',
 'premise': 'Roman Atwood . He is best known for his vlogs , where he posts '
            'updates about his life on a daily basis . His vlogging channel , '
            "`` RomanAtwoodVlogs '' , has a total of 3.3 billion views and "
            '11.9 million subscribers . He also has another YouTube channel '
            "called `` RomanAtwood '' , where he posts pranks .",
 'srl': {'hypothesis': {'annotations': [{'englishPropbank': {'frameName': 'be.01',
              

In [ ]:

import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
print(stopwords.words('english'))



['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
f1 = "test per vedere come va<\s>questa è una prova"

tokenized = tokenizer(f1, return_tensors='pt', padding='max_length', max_length=40)

print(tokenized)
print(tokenized["input_ids"].shape)
print(type(tokenized))

#out = model(**tokenized)

{'input_ids': tensor([[    0, 21959,   228,   748,   196,  2816,   283, 13205, 41552, 37457,
            29, 15698, 23627,   102,   952, 11423,   542,   102, 11570,   102,
             2,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}
torch.Size([1, 40])
<class 'transformers.tokenization_utils_base.BatchEncoding'>


In [ ]:

print(out.last_hidden_state.mean(dim=-1).squeeze().shape)

torch.Size([21])


In [41]:
class NLIDataset(Dataset):



    def __init__(self, data):

      self.encode_labels = {'CONTRADICTION': 0, 'NEUTRAL': 1, 'ENTAILMENT': 2}

      self.data = []

      for example in data:
          inputs = tokenizer(example["premise"], example["hypothesis"], return_tensors='pt')
          token_ids = inputs['input_ids'][0]
          sep_token_id = tokenizer.sep_token_id

          # Trovare la posizione del token di separazione
          sep_index = (token_ids == sep_token_id).nonzero(as_tuple=True)[0].tolist()[0]
          element = {
              "input": inputs,
              "label": self.encode_labels[example["label"]],
              "separator_index": sep_index
              }
          self.data.append(element)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

    def collate(self, batch):
        x = []
        y = []
        z = []
        attention_mask = []
        #print(batch)
        #breakpoint()
        # Iterare sull'array di dizionari una sola volta
        for d in batch:
            x.append(d['input']['input_ids'].squeeze())
            y.append(d['label'])
            z.append(d['separator_index'])
            attention_mask.append(d['input']['attention_mask'].squeeze())
       #breakpoint()
        # Convertire le liste di valori in tensori PyTorch
        x = pad_sequence(x, batch_first=True, padding_value=tokenizer.pad_token_id)
        attention_mask = pad_sequence(attention_mask, batch_first=True, padding_value=0)
        y = torch.tensor(y)
        z = torch.tensor(z)
        return x,attention_mask, y, z



    def get_dataloader(self, batch_size, pos_num):
        self.pos_num = pos_num
        return DataLoader(self, batch_size=batch_size, shuffle=True, collate_fn = self.collate)

















In [8]:
# prompt: create a torch model using roberta and a linear layer

class RobertaClassifier(nn.Module):
  def __init__(self, num_labels=3):
    super(RobertaClassifier, self).__init__()
    self.roberta = RobertaModel.from_pretrained('roberta-base')
    self.linear = nn.Linear(self.roberta.config.hidden_size, num_labels)

  def forward(self, input_ids, attention_mask):
    outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
    pooled_output = outputs.pooler_output
    logits = self.linear(pooled_output)
    return logits


In [42]:


dataset = NLIDataset(validation_set)


dataloader = dataset.get_dataloader(batch_size=32, pos_num=1)

model = RobertaClassifier()

batch = next(iter(dataloader))

model(batch[0], batch[1])

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tensor([[ 0.0798, -0.0058,  0.0531],
        [ 0.0781, -0.0083,  0.0441],
        [ 0.0795, -0.0226,  0.0402],
        [ 0.0710, -0.0083,  0.0479],
        [ 0.0697, -0.0217,  0.0390],
        [ 0.0773, -0.0005,  0.0327],
        [ 0.0785, -0.0074,  0.0525],
        [ 0.0791, -0.0153,  0.0619],
        [ 0.0834, -0.0076,  0.0606],
        [ 0.0729, -0.0161,  0.0349],
        [ 0.0724, -0.0175,  0.0429],
        [ 0.0768, -0.0089,  0.0442],
        [ 0.0603, -0.0185,  0.0448],
        [ 0.0696, -0.0148,  0.0436],
        [ 0.0866, -0.0151,  0.0480],
        [ 0.0801, -0.0220,  0.0551],
        [ 0.0783, -0.0167,  0.0507],
        [ 0.0709, -0.0171,  0.0432],
        [ 0.0858, -0.0013,  0.0641],
        [ 0.0639, -0.0060,  0.0479],
        [ 0.0721, -0.0044,  0.0459],
        [ 0.0819, -0.0074,  0.0496],
        [ 0.0915, -0.0203,  0.0600],
        [ 0.0570, -0.0137,  0.0363],
        [ 0.0693, -0.0127,  0.0430],
        [ 0.0798, -0.0184,  0.0524],
        [ 0.0735, -0.0174,  0.0480],
 